# $\color{teal}{\text{Compressive Sensing}}$ 
### $\color{gray}{\text{1. Introduction}}$

#### $\color{gray}{\text{1.1 Problem Setting}}$
In signal processing we are often interested in reconstructing a signal $x \in \mathbb{C}^N$ given some observation data $y \in \mathbb{C}^N$. By assuming linear measurements we can state the linear system of equations $Ax = y$. Here, the matrix $A \in \mathbb{C}^{m \times N}$ denotes the measurement process and is often referred to as the measurement matrix. Solving this linear system depends on the dimensions of $A$. If the number of measurements $m$ is at least as large as the signal length $N$ we can solve the well-defined system of equations. However, if we consider the case that $m<N$, i.e. $A$ is a wide matrix, we get an underdetermined linear system. Due to the ill-posedness of the underdetermined system $Ax=y$, the original problem becomes an infinite dimensional problem with infinitely many solutions. Compressive Sensing (CS) introduces certain assumptions on the signal $x$ and provides the theory to recover $x$ when the system $Ax=y$ is underdetermined. The underlying assumption is sparsity, restricting solutions to vectors that have few nonzero entries.

####  $\color{gray}{\text{1.1 Sparsity}}$
A vector is $s$-sparse if it has at most $s$ nonzero elements. Using the notion of the $\mathcal{l}_0$-norm denoted by $||x||_0$, we count the number of nonzero elements of the vector $x \in \mathbb{C}^N$. As it turns out, many real-world signals become sparse when properly transformed - this is the case in image compression. In the following code cell we compress an image by applying the Discrete Wavelet Transform (DWT) and reconstruct the image with a small amount of wavelet coefficients. For the reconstruction we used only the strongest wavelet coefficients as all other coefficients were set to zero. 

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

path = r".\images\kiz_kulesi.jpg"

image = Image.open(path).convert("L")
data = np.asarray(image)
plt.imshow(data, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
#transform image to numpyarray and sparsify it
path = r".\images\pokeball.png"

image = Image.open(path).convert("L")
np_data = np.asarray(image)
plt.imshow(np_data, cmap='gray', vmin=0, vmax=255)
plt.show()

[rows,clmns] = np_data.shape
index_row = np.arange(rows)
index_clm = np.arange(clmns)

sparse_A = np.zeros((rows,50))
sparse_A[:,index_clm] = np_data[:,index_clm]

plt.imshow(sparse_A, cmap='gray', vmin=0, vmax=255)
plt.show()

In [ ]:
#https://pywavelets.readthedocs.io/en/latest/
import pywt

titles = ['Approximation', ' Horizontal detail',
          'Vertical detail', 'Diagonal detail']
wavelet = pywt.Wavelet('db1')

# Multilevel decomposition of the input matrix
coeffs2 = pywt.dwt2(arr, wavelet, )
cA2, (cH2, cV2, cD2), (cH1, cV1, cD1) = coeffs

# Concatenate the level-2 submatrices into a big one and plot
x_house_star_wav = np.bmat([[cA2, cH2], [cV2, cD2]])
LL, (LH, HL, HH) = coeffs2
fig = plt.figure(figsize=(12, 3))
for i, a in enumerate([LL, LH, HL, HH]):
    ax = fig.add_subplot(1, 4, i + 1)
    ax.imshow(a, interpolation="nearest", cmap=plt.cm.gray)
    ax.set_title(titles[i], fontsize=10)
    ax.set_xticks([])
    ax.set_yticks([])

fig.tight_layout()
plt.show()

Obviously, we exploted the compressibility of natural images. In practice we usually deal with vectors that are not exactly $s$-sparse. Due to their compressibility we can still find a suitable approximation which is quantified by the error of the best $s$-term approximation 

$$\sigma_s(x)_p = \underset{||z||_0 \leq s}{\mathrm{inf}} ||x-z||_p$$

where $z$ is a sparse vextor. One might be interested in how many measurements we need to recover all $s$-sparse vectors x $-$ spoiler: $m = 2s$ measurements.

####  $\color{gray}{\text{1.2 The } \mathcal{l}_0 \text{-Minimization Problem}}$
With this insight, the first intuitive approach is to search for the sparsest vector satisfying $Az=y$. This can be formulated as the $\mathcal{l}_0$-minimization problem 

$$ \text{minimize } ||z||_0 \;\;\; \text{subject to } Az=y$$

which we denote as $\color{green}{(P_0)}$. In practical applications we have to deal with noisy measurements which may affect the stability of our algorithms. By tolerating an error value $\eta$ we can introduced the less strict optimization problem

$$ \text{minimize } ||z||_0 \;\;\; \text{subject to } ||Az-y||_2 \leq \eta$$

as $\color{green}{(P_{0,\eta})}$. Unfortunately, both problems ar NP-hard which means that there are no efficient algorithms to solve them and thus unhelpful in practice. The corresponding proof is based on the exact cover by $3$-sets problem, which is NP-complete. At first glance it is rather surprising that efficient recovery algorithms do exist. These can be categorized into optimization methods, greedy methods and thresholding-based methods.

#### $\color{gray}{\text{1.3 Basis Pursuit}}$
Instead of solving the $\color{green}{(P_0)}$ we can consider a manageable approximation given as 

$$ \text{minimize } ||z||_1 \;\;\; \text{subject to } Az=y$$

which we denote with $\color{green}{(P_1)}$. One important property of the $\mathcal{l}_1$-norm is that it favors sparse solutions. It turns out that the basis pursuit (also called $\mathcal{l}_1$-minimization) is a convex optimization problem, for which efficient solving algorithms exist. 

#### $\color{gray}{\text{Iterative Hard Thresholding}}$
The so-called restricted isometry property (RIP) of matrices gives a hint on whether the sparse recovery algorithm will succeed or not. It is NP-hard to show that a given matrix fullfilles the RIP, but fortunately a theorem assures that subgaussian random matrices with i.i.d. entries satisfy the RIP with high probability. Therefore, we are going to use the numpy function `random.randn()` to generate the corresponding measurement matrix $A$.


The iterative hard thresholding (IHT) algorithm is based on non-convex optimization. Hence, it converges to local minima and depends on the initialization of `x_0` (commomly with the all-zero vector). In following we will implement the general setup of CS, i.e. generate the measurement matrix $A$ and the measurements $y$ based on the true vector `x_true` that is randomly generated. 

In [ ]:
import numpy as np
import random
import math


#iht:
m = 300
N = 1000
s = 10

#omp
#m = 200
#N = 175
#s = 30

#ground truth
x_true = np.zeros(N)
indices = random.sample(range(N),s) #randomly sample s out of 1000 indices
x_true[indices] = np.random.rand(s)

#random measurement matrix
A = np.random.randn(m,N)
A = A/math.sqrt(m)
#simulated measurements
y = A @ x_true

In each iteration of the IHT algorithm we calculate 

$$x^{(k+1)} = \mathcal{H}_s\{x^{(k)} + A^{*}(y-Ax^{(k)})\}$$

where $\mathcal{H}_s$ denotes the hard thresholding operator of order $s$ and $A^{*}$ the hermitian of $A$. By applying $\mathcal{H}_s$ we take only the $s$ entries with the largest absolute values and set the other ones to zero, thus ensuring the sparsity $s$ of $x^{(k+1)}$. 

In [ ]:
def H_s(x_tmp,s):
    """Hard-thresholding operator: keep the s largest entries of z and set the other ones to zero."""
    N = x_tmp.shape[0]
    x_new = np.zeros(N)
    indices = x_tmp.argsort()[-s:]
    x_new[indices] = x_tmp[indices]
    return x_new

In [ ]:
def iht(A, y, s, iters):
    """Implementation of the IHT."""
    [m,N] = A.shape
    x_0 = np.zeros(N)
    
    #hermitian of A
    A_H = A.conjugate().transpose()
    
    x_iters = []
    err_iters = np.zeros(iters)
    x_hat = np.zeros(N)
    
    from tqdm import tqdm
    for i in tqdm(range(iters)):
        if i==0:
            x_k = x_0
        else:
            x_k = H_s(x_k + A_H@(y - A@x_k),s)
            #@TODO: iteratively plot x_k and x_true
            #print(f"x_hat_{i}: {[val for val in x_k if val!=0]}\n")
            x_hat = x_k
        x_iters.append(x_k)
        
        err = np.linalg.norm(A@x_k-y,2) #l2-error
        err_iters[i] = err
        print(f"err iteration {i}: {err}")
    return x_hat, x_iters, err_iters

Run the iterative hard thresholding algorithm.

In [ ]:
n_iters = 100
x_hat, x_iters, err_iters = iht(B,z,10,n_iters)

In [ ]:
err = np.linalg.norm(x_hat-x_true,2) #l2-error
print(err)

#### $\color{gray}{\text{Orthogonal Matching Pursuit}}$
This algorithm is a representant of so-called greedy methods. Since the original $\color{green}{(P_{0,\eta})}$ is NP-hard, we use the OMP as an approximation method. It iteratively constructs the support set of the reconstructed sparse vector by extending the current support set with an additional index at each iteration. One drawback of this algorithm is that if a wrong index has been selected, it will remain in all subsequent index sets. The initial support set shall be the empty set and $x^{(0)}$ shall be equal to the all-zero vector. Further, the residual $r^{(0)}$ is chosen to be $y$.

Each iteration of the OMP algorithms covers mainly two steps denoted by `OMP_1` and `OMP_2` and mathematically expressed as

$$ j_{k+1} = \underset{j \in [N]}{\mathrm{argmax}} \{|a^{*}_j \cdot r^{(k)}|\}, \;\;\; S^{(k+1)}=S^{(k)} \cup \{j_{k+1}\}$$
and 
$$ x^{(k+1)} = \underset{z \in \mathbb{C}^{N}}{\mathrm{argmin}} \{||y-Az||_2\} \;\;\; \text{s.t. supp}(z) \subset S^{(k+1)} $$

We further introduce the residual term $r^{k+1} = y - Ax^{(k+1)}$, which can be understood as an error term. The desired output of the algorithm is an $\bar{n}$-sparse vector, where $\bar{n}$ denotes the number of iterations needed for the recovery of $x$. Since `OMP_2` is a least squares problem, we can calculate $x^{(k+1)}$ using the pseudoinverse $A^{\dagger}_{S^{(k+1)}} \in \mathbb{C}^{N \times m}$ that is restricted to the set $S^{(k+1)}$ via

$$ x^{(k+1)}=A^{\dagger}_{S^{(k+1)}}y = (A^{*}_{S^{(k+1)}}A_{S^{(k+1)}})^{-1}A^{*}_{S^{(k+1)}}y.$$

Restricting a matrix on a support set means that that we take all columns indicated in the support set and set the other ones to zero.

In [ ]:
import numpy as np
import random
import math

#omp
m = 200
N = 175
s = 30

#ground truth
x_true = np.zeros(N)
indices = random.sample(range(N),s) #randomly sample s out of 1000 indices
x_true[indices] = np.random.rand(s)

#random measurement matrix
A = np.random.randn(m,N)
A = A/math.sqrt(m)
#simulated measurements
y = A @ x_true

In [ ]:
def get_max_index(residual,matrix,n_clmns):
    matrix_T = matrix.T
    dot_products =[np.dot(matrix_T[i].conjugate(),residual) for i in range(n_clmns)]
    abs_vals = np.absolute(dot_products)
    return np.argmax(abs_vals)

In [ ]:
def restrict_on_support(matrix,support_set):
    """Restrict matrix on support set by copying all elements in the columns indicated by 
    support_set and setting the other ones to zero."""
    
    [rows,clms] = matrix.shape
    matrix_supp = np.zeros((rows,clms), dtype='float32')
    matrix_supp[:,support_set] = matrix[:,support_set]
    return matrix_supp

In [ ]:
def omp(A,y,iters):
    """Implementation of the Orthogonal Matching Pursuit."""   
    from numpy.linalg import pinv
    
    [m,N] = A.shape
    #Initializations
    x_iters = []
    support_set = []
    err_iters = np.zeros(iters)
    x_0 = np.zeros(N)
    r_0 = y
    
    #Iteration
    for i in range(iters):
        if i==0:
            residual = r_0
        print(f"residual: \n {residual[50]}")
        index = get_max_index(residual,A,N)
        print(f"index: {index}")
        support_set.append(index)
        print(support_set)
        
        A_r = restrict_on_support(A,support_set)       
        x_new = np.linalg.pinv(A_r)@y
        
        print(f"x_new: \n {sum(x_new)}")
        residual = y - A@x_new
        x_iters.append(x_new)
        
        err = np.linalg.norm(A@x_new-y,2) #l2-error
        err_iters[i] = err
        print(f"err iteration {i}: {err}")
    
    return x_new, x_iters, err_iters

In [ ]:
n_iters = 100
x_hat, x_iters, err_iters = omp(A,y,n_iters)

In [ ]:
err = np.linalg.norm(x_hat-x_true,2) #l2-error
print(err)

#### $\color{gray}{\text{Algorithms}}$
Alternative reconstruction
methods include greedy-type methods such as orthogonal matching pursuit, as well
6 1 An Invitation to Compressive Sensing
as thresholding-based methods including iterative hard thresholding (Foucart p.6)

####  $\color{gray}{\text{References}}$
[1] Mathematical Introducation to Compressive Sensing, Simon Foucart, Holger Rauhut